In [14]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [15]:
import awswrangler as wr

import mlflow

# Para que funcione, en todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [16]:
mlflow_server = "http://localhost:5001"

mlflow.set_tracking_uri(mlflow_server)

In [26]:
import datetime
import warnings

import matplotlib.pyplot as plt
from mlflow.models import infer_signature
from neuralprophet import NeuralProphet
import numpy as np
import optuna
from sklearn.metrics import mean_absolute_error

from mlflow_aux import get_or_create_experiment
from neural_prophet_wrapper import NeuralProphetWrapper
from optuna_aux import champion_callback, objective
from trainer_aux import set_trainer

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

warnings.filterwarnings("ignore")

In [29]:
raw_data_path = "s3://data/raw/trx_recaudacion_km_empresa_54_2020_2025.xlsx"
sheets = wr.s3.read_excel(raw_data_path, sheet_name=None)
lineas = sheets.keys()

N_TRIALS = 30
EPOCHS = 100
BATCH_SIZE = 128
regresores = ["km"]

for linea in lineas:

    # Leer datos de entrenamiento y validación
    df_train = wr.s3.read_csv(f"s3://data/processed/{linea}/train.csv", parse_dates=["ds"])
    df_test = wr.s3.read_csv(f"s3://data/processed/{linea}/test.csv", parse_dates=["ds"])

    experiment_id = get_or_create_experiment(f"Línea {linea}")

    run_name_parent = "best_hyperparam_" + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')
    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):

        # Optimización con Optuna
        study = optuna.create_study(direction="minimize")
        study.optimize(
            lambda trial: objective(trial, df_train.copy(), df_test.copy(),
                                    epochs=EPOCHS, batch_size=BATCH_SIZE, regresores=regresores),
            n_trials=N_TRIALS,
            callbacks=[champion_callback]
        )

        # Log de mejores hiperparámetros
        mlflow.log_params(study.best_params)
        mlflow.log_metric("best_train_mae", study.best_value)

        mlflow.set_tags({
            "project": f"Línea {linea}",
            "optimizer_engine": "optuna",
            "model_family": "neuralprophet",
            "feature_set_version": 1,
        })

        # Crear modelo con los mejores hiperparámetros
        model = NeuralProphet(
            n_changepoints=study.best_params["n_changepoints"],
            trend_reg=study.best_params["trend_reg"],
            learning_rate=study.best_params["learning_rate"],
            seasonality_mode=study.best_params["seasonality_mode"],
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
        )
        set_trainer(model, EPOCHS)

        # Agregar regresores
        for reg in regresores:
            model.add_future_regressor(reg)
        
        # Entrenamiento
        model.fit(df_train, freq='D')

        # Predicción sobre validación
        forecast_test = model.predict(df_test)
        # Seleccionar solo columnas numéricas
        num_cols = forecast_test.select_dtypes(include=[np.number]).columns

        # Reemplazar negativos por 0 solo en columnas numéricas
        forecast_test[num_cols] = forecast_test[num_cols].clip(lower=0)

        # Cálculo de métricas de error
        mae = mean_absolute_error(df_test['y'].values, forecast_test['yhat1'].values)
        mlflow.log_metric("test_mae", mae)

        # Guardar artefacto del modelo
        artifact_path = f"linea_{linea}_model"
        sample_input = df_train[["ds", "y"] + regresores].copy().tail(30)
        sample_output = model.predict(sample_input)[["ds", "yhat1"]]
        signature = infer_signature(sample_input, sample_output)

        mlflow.pyfunc.log_model(
            python_model=NeuralProphetWrapper(model),
            code_paths=["./neural_prophet_wrapper.py"],
            name=artifact_path,
            signature=signature,
            registered_model_name=f"linea_{linea}_model_dev",
            metadata={"model_data_version": 1},
        )

        # --- Forecast en train y test ---
        forecast_train = model.predict(df_train)
        # Seleccionar solo columnas numéricas
        num_cols = forecast_train.select_dtypes(include=[np.number]).columns

        # Reemplazar negativos por 0 solo en columnas numéricas
        forecast_train[num_cols] = forecast_train[num_cols].clip(lower=0)

        fig, ax = plt.subplots(figsize=(12, 6))

        # Valores reales de entrenamiento
        ax.plot(df_train["ds"], df_train["y"], label="Train Real", color="blue")

        # Predicciones en entrenamiento
        ax.plot(forecast_train["ds"], forecast_train["yhat1"], 
                label="Train Pred", color="cyan", linestyle="--")

        # Valores reales de test
        ax.plot(df_test["ds"], df_test["y"], label="Test Real", color="black")

        # Predicciones en test
        ax.plot(forecast_test["ds"], forecast_test["yhat1"], 
                label="Test Pred", color="red", linestyle="--")

        ax.set_title(f"Train/Test Forecast - Línea {linea}")
        ax.set_xlabel("Fecha")
        ax.set_ylabel("y")
        ax.legend()

        # Guardar en MLflow
        plot_path = f"/tmp/forecast_train_test_{linea}.png"
        fig.savefig(plot_path)
        plt.close(fig)

        mlflow.log_artifact(plot_path, artifact_path="plots")

        # Componentes
        #model.plot_components(forecast, plotting_backend="matplotlib")
        #fig_components = plt.gcf()
        #components_path = f"/tmp/components_{linea}.png"
        #fig_components.savefig(components_path)
        #plt.close(fig_components)
        #mlflow.log_artifact(components_path, artifact_path="plots")

    mlflow.end_run()

INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
INFO	Task(Task-3) NP.config:configure.py:__post_init__()- Note: Trend changepoint regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING	Task(Task-3) NP.forecaster:forecaster.py:fit()- When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.901% of the data.
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Major frequency D corresponds to 99.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.


Epoch 100: 100%|██████████| 100/100 [00:00<00:00, 4611.51it/s, loss=0.0962, v_num=1641, MAE=9.37e+3, RMSE=1.27e+4, Loss=0.0937, RegLoss=0.000]  

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.603% of the data.
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Major frequency D corresponds to 99.603% of the data.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.603% of the data.
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Major frequency D corresponds to 99.603% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Defined frequency is equal to major frequency - D


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 418.22it/s]

...

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO	Task(Task-3) NP.df_utils:df_utils.py:return_df_in_original_format()- Returning df with no ID column


Registered model 'linea_504I_model_dev' already exists. Creating a new version of this model...
2025/08/23 17:30:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linea_504I_model_dev, version 13
Created version '13' of model 'linea_504I_model_dev'.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.901% of the data.
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Major frequency D corresponds to 99.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.901% of the data.
INFO	Task(Task-3) NP.df_utils:df_utils.py:_infer_frequency()- Major frequency D corresponds to 99.901% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined fr

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 288.43it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO	Task(Task-3) NP.df_utils:df_utils.py:return_df_in_original_format()- Returning df with no ID column



🏃 View run best_hyperparam_2025/08/23-17:28:56" at: http://localhost:5001/#/experiments/9/runs/024e474a51e34cadba98396d384efc1a
🧪 View experiment at: http://localhost:5001/#/experiments/9
